In [1]:
import pandas as pd
import numpy as np
import os
import scipy
import sqlite3
import tensorflow as tf
import string
from sklearn.metrics import accuracy_score

In [2]:
# create our data set
curr_path = os.path.realpath('')
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")

###@@@@@ 
#         NORMALIZE and add loan_amnt and annual_inc back later 
###@@@@

# this line includes loan amnt & annual_inc. Makes accuracy go to 23%~
# df_tf = pd.read_sql_query("select loan_status, term, int_rate, emp_length, home_ownership, verification_status, dti, annual_inc, loan_amnt from loan where id in (select id from loan order by random() AND emp_length != 'n/a' OR loan_status =='Fully Paid' OR loan_status == 'Charged Off' OR loan_status == 'Default');", conn)
df_tf = pd.read_sql_query("select loan_status, term, int_rate, emp_length, home_ownership, verification_status, dti from loan where id in (select id from loan order by random() AND emp_length != 'n/a' OR loan_status =='Fully Paid' OR loan_status == 'Charged Off' OR loan_status == 'Default');", conn)



# df_desc = pd.read_sql_query("select loan_amnt, loan_status, sub_grade from loan where desc != 'NULL' AND loan_status !='Current' AND loan_status !='Late (31-120 days)' AND loan_status != 'Late (16-30 days)'", conn)
print (len(df_tf))
df_tf = df_tf.sample(frac=1).reset_index(drop=True)


887383


In [3]:
data = []
results = []
for i in range(100000):  # data set of 10000 for faster run time. Replace with range(len(df_tf)) later
    # one hot encoding for loan status. Or 0, 1, 2
    loan_status = 0 if df_tf.iloc[i][0] == 'Fully Paid' else 1 # bad if not fully paid
    
    # 0 if term is 36 months, 1 if 60 months
    term = 0 if df_tf.iloc[i][1] == ' 36 months' else 1
    
    # change interest rate from str to float
    int_rate = float(df_tf.iloc[i][2].replace('%', ''))
    
    # get employment length
    txt = ''.join([c for c in df_tf.iloc[i][3].lower() if not c in set(string.punctuation)])
    emp_length_list = ([int(s) for s in txt.split() if s.isdigit()])
    emp_length = 0 if emp_length_list == [] else emp_length_list[0]

    # 0 if renting, 1 if owner
    ownership = 0 if df_tf.iloc[i][4] == 'RENT' else 1
    
    # verification sttus, 0 for not verified, 1 for source verified and 2 for verified
    #verification = 0 if df_tf.iloc[i][5] == 'Not Verified' else 1 if df_tf.iloc[i][5] == 'Source Verified' else 2
    verification = 0 if df_tf.iloc[i][5] == 'Not Verified' else 1

    # append in feature vector
   # data.append([term, int_rate, emp_length, ownership, verification, df_tf.iloc[i][6], df_tf.iloc[i][7], df_tf.iloc[i][8]])
    data.append([term, int_rate, emp_length, ownership, verification, df_tf.iloc[i][6]])
         
    # add loan status to results 
    results.append(loan_status)

print (data[-1:])
print (results[-1:])
X_train = data[:int(len(data)/2)]
y_train = results[:int(len(results)/2)]

X_val = data[int(len(data)/2): int(len(data)*0.75)]
y_val = results[int(len(results)/2): int(len(results)*0.75)]

X_test = data[int(len(data)*.75):]
y_test = results[int(len(results)*.75):]

conn.close()

[[0, 10.15, 10, 1, 1, 15.880000000000001]]
[1]


In [4]:
X_train_arr = np.array(X_train)
y_train_arr = np.array(y_train)
#print (X_train)
#print (y_train)
#print (X_train_arr)
#print (y_train_arr)

In [7]:
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train_arr)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[100,50], n_classes=2,feature_columns=feature_cols)
# dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) 
dnn_clf.fit(X_train_arr, y_train_arr, batch_size=10, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a1316b5c0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving 

/Users/noellin/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/cv/j7y0jf0x0gbc39xntlnng44c0000gn/T/tmpkkf9cpwe/model.ckpt.
INFO:tensorflow:loss = 3.36177, step = 1
INFO:tensorflow:global_step/sec: 808.845
INFO:tensorflow:loss = 0.552753, step = 101 (0.126 sec)
INFO:tensorflow:global_step/sec: 812.045
INFO:tensorflow:loss = 0.685137, step = 201 (0.123 sec)
INFO:tensorflow:global_step/sec: 549.502
INFO:tensorflow:loss = 0.455403, step = 301 (0.182 sec)
INFO:tensorflow:global_step/sec: 625.555
INFO:tensorflow:loss = 0.478873, step = 401 (0.160 sec)
INFO:tensorflow:global_step/sec: 715.435
INFO:tensorflow:loss = 0.44298, step = 501 (0.139 sec)
INFO:tensorflow:global_step/sec: 784.967
INFO:tensorflow:loss = 0.599746, step = 601 (0.127 sec)
INFO:tensorflow:global_step/sec: 762.509
INFO:tensorflow:loss = 0.598074, step = 701 (0.131 sec)
INFO:tensorflow:global_step/sec: 609.767
INFO:tensorflow:loss = 0.554206, step = 801 (0.167 sec)
INFO:tensorflow:globa

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x1a19b6f5f8>, 'hidden_units': [100, 50], 'feature_columns': (_RealValuedColumn(column_name='', dimension=6, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x111116510>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [8]:
X_val_arr = np.array(X_val)
y_val_arr = np.array(y_val)
y_pred = dnn_clf.predict_classes(X_val_arr, as_iterable=False)
print('')
print('Accuracy: ' + str(accuracy_score(y_val_arr, y_pred)))

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/cv/j7y0jf0x0gbc39xntlnng44c0000gn/T/tmpkkf9cpwe/model.ckpt-1000

Accuracy: 0.76924


/Users/noellin/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b
